# Complete pipeline

In [1]:
from inf_functions import *
from class_functions import *
import matplotlib.pyplot as plt
import sklearn
import os 

os.environ['CUDA_VISIBLE_DEVICES']='2'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2025-10-30 10:45:48.698604: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1761817550.214700  915376 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46131 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:61:00.0, compute capability: 8.6


## Low mass

We select the first event from the low mass generated artificial dataset (*'/home/alberto_sinigaglia/gaia/CNN_low_mass/Test/Signal/mass1-12_mass2-10_sample1.hdf'*)

In [23]:
low_mass_example = '/home/alberto_sinigaglia/gaia/CNN_low_mass/Test/Signal/mass1-12_mass2-10_sample1.hdf'

### 1. Classification 

Firstly, we classify the selected event.

**questa è da sistemare**

In [11]:
root_dir = "/home/alberto_sinigaglia/gaia"
mass_range = "CNN_low_mass"
test_path = f"{root_dir}/{mass_range}_test.npz"

In [13]:
with np.load(test_path, allow_pickle=True) as data:
    X_test = data["X"]
    y_test = data["y"]

batch_size = 64
test_dataset = make_dataset(X_test, y_test, classification=True, batch_size=batch_size, shuffle=False)

In [24]:
class_model_path = f"class_models/{mass_range}_best_model.keras"
class_model = tf.keras.models.load_model(class_model_path)

y_test_pred = class_model.predict(X_test)
pred = (y_test_pred > 0.4).astype(int)

## our example
label = pred[0]

if label==1:
    print('The selected strain corresponds to a GW event.')
else:
    print('The selected strain corresponds to a noise event.')

41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step
The selected strain corresponds to a noise event.


### 2. Regression

Then, we infere the chirp mass and mass ratio.

In [19]:
with np.load(test_path, allow_pickle=True) as data:
    X_test = data["X"]
    y_test = data["y"]

mask = y_test[:,0] == 1
X_test, y_test= X_test[mask], y_test[mask]

# create the dataset 
batch_size = 64
test_dataset, test_scaler = make_dataset(X_test, y_test, classification=False, batch_size=batch_size, shuffle=False)

In [ ]:
reg_model_path = f"reg_models/{mass_range}_best_model.keras"
reg_model = tf.keras.models.load_model(reg_model_path)

y_pred = reg_model.predict(X_test)
y_pred = test_scaler.inverse_transform(y_pred)

q_pred = y_pred[:,1]

M_true = y_test[:,1].astype(np.float64)
M_pred = y_pred[:,0]
rmse = np.sqrt(np.mean((M_pred - M_true)**2))

41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step


In [22]:
filename = low_mass_example.split('/')[-1]  # 'mass1-12_mass2-10_sample1.hdf'
parts = filename.replace('.hdf', '').split('_') 
m1_true = float(parts[0].split('-')[1])
m2_true = float(parts[1].split('-')[1])
q_cnn = q_pred[0]
M_cnn = M_pred[0]
M_err = rmse

print(f"True parameters: m1 = {m1_true}, m2 = {m2_true}, q = {m2_true/m1_true:.2f}, M = {(m1_true*m2_true)**(3/5)/(m1_true+m2_true)**(1/5):.2f}")
print(f"CNN predicted parameters: q = {q_cnn:.2f}, M = {M_cnn:.2f}")
print(f"RMSE on chirp mass prediction: {M_err:.2f}")

True parameters: m1 = 12.0, m2 = 10.0, q = 0.83, M = 9.53
CNN predicted parameters: q = 0.71, M = 9.72
RMSE on chirp mass prediction: 1.63


### 3. Parameter inference

In [35]:
m1_chain, m2_chain, results = estimate_masses(M_cnn, q_cnn, M_err)

100%|██████████| 53000/53000 [00:00<00:00, 154541.43it/s]


MCMC for m1:
  Acceptance rate: 0.821
  Mean: 13.59 M☉
  Median: 13.45 M☉
  Std: 1.96 M☉


100%|██████████| 53000/53000 [00:00<00:00, 110540.26it/s]

MCMC for m2:
  Acceptance rate: 0.646
  Mean: 11.09 M☉
  Median: 10.89 M☉
  Std: 0.86 M☉


In [ ]:
import seaborn as sns

# Converti in array
m1 = m1_chain.flatten()
m2 = m2_chain.flatten()

sns.set(style="white", context="talk")

plt.figure(figsize=(8,7))

# KDE contour
sns.kdeplot(
    x=m1, 
    y=m2,
    fill=True,
    cmap="Blues",
    levels=15,
    thresh=0.01,
    alpha=0.8
)

# Scatter point true values
plt.scatter(m1_true, m2_true, color="red", marker="*", s=200, label="Valori veri")

# Scatter point medians
plt.scatter(m1_median, m2_median, color="gold", edgecolor="black", marker="o", s=120, label="Mediana MCMC")

# Guide lines
plt.axvline(m1_true, color="red", linestyle="--", alpha=0.8)
plt.axhline(m2_true, color="red", linestyle="--", alpha=0.8)

plt.axvline(m1_median, color="gold", linestyle="--", alpha=0.8)
plt.axhline(m2_median, color="gold", linestyle="--", alpha=0.8)

plt.xlabel("Massa primaria $m_1$ (M$_\\odot$)")
plt.ylabel("Massa secondaria $m_2$ (M$_\\odot$)")
plt.title("Posterior joint KDE ($m_1, m_2$) con verità e mediane")
plt.legend()

sns.despine()
plt.tight_layout()
plt.show()


In [ ]:
from scipy.stats import gaussian_kde

# ---- USER INPUT ----
# Replace these with your real data
m1 = m1_chain.flatten()
m2 = m2_chain.flatten()
m1_med = m1_median
m2_med = m2_median

# ---- KDE 2D ----
kde = gaussian_kde(np.vstack([m1, m2]))

xmin, xmax = m1.min(), m1.max()
ymin, ymax = m2.min(), m2.max()

xx, yy = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]
positions = np.vstack([xx.ravel(), yy.ravel()])
f = kde(positions).reshape(xx.shape)

# ---- Layout ----
fig = plt.figure(figsize=(8,8))
grid = plt.GridSpec(4,4, hspace=0.05, wspace=0.05)

ax_main  = fig.add_subplot(grid[1:4, 0:3])
ax_top   = fig.add_subplot(grid[0,     0:3], sharex=ax_main)
ax_right = fig.add_subplot(grid[1:4,     3], sharey=ax_main)

# ---- Joint contour ----
ax_main.contourf(xx, yy, f, levels=15, alpha=0.6)
ax_main.contour(xx, yy, f, colors='k', linewidths=0.8)

# True point
ax_main.scatter(m1_true, m2_true, color="red", s=120, marker="*", label="True values")
# Median point
ax_main.scatter(m1_med, m2_med, color="gold", edgecolor="black", s=100, label="Posterior median")

# Lines
ax_main.axvline(m1_true, color="red", linestyle="--")
ax_main.axhline(m2_true, color="red", linestyle="--")
ax_main.axvline(m1_med, color="gold", linestyle="--")
ax_main.axhline(m2_med, color="gold", linestyle="--")

# --- Marginal: m1 ---
ax_top.hist(m1, bins=40, density=True, alpha=0.7, color="steelblue")
ax_top.axvline(m1_true, color="red", linestyle="--")
ax_top.axvline(m1_med, color="gold", linestyle="--")
ax_top.set_ylabel("PDF")
plt.setp(ax_top.get_xticklabels(), visible=False)

# --- Marginal: m2 ---
ax_right.hist(m2, bins=40, density=True, alpha=0.7, orientation="horizontal", color="steelblue")
ax_right.axhline(m2_true, color="red", linestyle="--")
ax_right.axhline(m2_med, color="gold", linestyle="--")
ax_right.set_xlabel("PDF")
plt.setp(ax_right.get_yticklabels(), visible=False)

# --- Styling ---
ax_main.set_xlabel(r"Primary mass $m_1$ ($M_\odot$)")
ax_main.set_ylabel(r"Secondary mass $m_2$ ($M_\odot$)")
ax_main.legend(loc="upper right")
ax_main.grid(alpha=0.3, linestyle="--")
plt.suptitle("Joint Posterior for $m_1$ and $m_2$", fontsize=16)

plt.tight_layout()
plt.show()